In [78]:
import numpy as np

time: 1.94 ms


In [73]:
%load_ext cython
%load_ext autotime

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.81 ms


In [74]:
def f(x):
    return x**2-x

def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a + dx * i)
    return s * dx
    

time: 3.39 ms


In [75]:
integrate_f(0, 10, 1000000)

283.3328833334909

time: 333 ms


In [76]:
%%cython

def f(x):
    return x**2-x

def integrate_f(a, b, N):
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a + dx * i)
    return s * dx


integrate_f(0, 10, 1000000)

time: 3.26 ms


In [77]:
integrate_f(0, 10, 1000000)

283.3328833334909

time: 231 ms


In [5]:
%%cython

time: 2.13 ms


UsageError: %%cython is a cell magic, but the cell body is empty.


# 5 reasons to use cython instead python

## 1: for using cython you could stay code as it is

## 2: you could use strict types

## 3: you could use functions from C and C++ which haven't bindings to python

For me it was CUDA functions in OpenCV

## 4: you could make your code faster very well

## 5: you can use a simple mechanism to do effective multithreading behaviour

## Conclusion

In [82]:
!pip3 freeze

absl-py==0.2.2
apturl==0.5.2
astor==0.7.1
astroid==2.0.4
backcall==0.1.0
beautifulsoup4==4.4.1
bleach==3.0.2
Brlapi==0.6.4
chardet==2.3.0
command-not-found==0.3
cycler==0.9.0
Cython==0.23.4
decorator==4.0.6
defer==1.0.6
defusedxml==0.5.0
docutils==0.12
entrypoints==0.2.3
flake8==3.5.0
gast==0.2.0
grpcio==1.13.0
h5py==2.7.1
html5lib==0.999
ipykernel==5.1.0
ipython==2.4.1
ipython-genutils==0.2.0
ipywidgets==7.4.2
isort==4.3.4
jedi==0.13.1
Jinja2==2.10
jsonschema==2.6.0
jupyter==1.0.0
jupyter-client==5.2.3
jupyter-console==6.0.0
jupyter-core==4.4.0
Keras==2.1.3
language-selector==0.1
lazy-object-proxy==1.3.1
lightdm-gtk-greeter-settings==1.2.1
louis==2.6.4
lxml==3.5.0
Mako==1.0.3
Markdown==2.6.6
Markups==1.0.1
MarkupSafe==0.23
mate-tweak==3.5.10
matplotlib==1.5.1
mccabe==0.6.1
mgen==1.2.0
mistune==0.8.4
moderngl==5.4.2
mpmath==0.19
nbconvert==5.4.0
nbformat==4.4.0
notebook==5.7.0
numexpr==2.4.3
numpy==1.11.0
onboard==1.2.0
pandas==0.17.1
pandocfilters==1.4.2
parso==0.3.1
pbr==5.1.1
pep8==